In [1]:
!wget https://github.com/alexmk7/nlp-2024/raw/refs/heads/main/data/news.txt.gz

--2025-01-08 19:28:16--  https://github.com/alexmk7/nlp-2024/raw/refs/heads/main/data/news.txt.gz
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexmk7/nlp-2024/refs/heads/main/data/news.txt.gz [following]
--2025-01-08 19:28:16--  https://raw.githubusercontent.com/alexmk7/nlp-2024/refs/heads/main/data/news.txt.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7700055 (7.3M) [application/octet-stream]
Saving to: ‘news.txt.gz’

news.txt.gz         100%[===================>]   7.34M  19.4MB/s    in 0.4s    

2025-01-08 19:28:18 (19.4 MB/s) - ‘news.txt.gz’ saved [7700055/7700055]



In [171]:
import gzip
import pandas as pd
import numpy as np

In [3]:
with gzip.open("news.txt.gz", "rt", encoding="utf-8") as f:
    lines = []
    for line in f:
        lines.append(line.strip().split("\t"))

In [4]:
df = pd.DataFrame(lines, columns=["Label", "Title", "Text"])

In [5]:
df

,Label,Title,Text
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г..."
3,economics,Цена нефти WTI снизилась после публикации данн...,Цена американской нефти WTI на лондонской бирж...
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"Сбербанк выставил на продажу долги по 21,4 тыс..."
...,...,...,...
9995,economics,Бывший пропагандист Буша нашел работу в торгов...,Бывший советник президента США Джорджа Буша по...
9996,travel,Россияне предпочли смартфон любимому человеку ...,Портал онлайн-бронирования отелей Hotels.com о...
9997,life,Потерявшаяся пять недель назад американка найд...,"В США два туриста обнаружили в лесу женщину, п..."
9998,media,"НТВ покажет решающие матчи ""Зенита"" и ЦСКА в Л...",Телеканал НТВ покажет в прямом эфире решающие ...


# Векторизация

In [285]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
from nltk.corpus import stopwords
nltk.download('stopwords')

from pymystem3 import Mystem

from tqdm.notebook import tqdm
tqdm.pandas()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [292]:
stop_words = set(stopwords.words('russian'))
regex_tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer("russian")
mystem = Mystem()

def preprocess(text):
    regexed_text = ' '.join(regex_tokenizer.tokenize(text.lower()))
    lemmatized_text = mystem.lemmatize(regexed_text)
    stopworded_lemmatized_text = ''.join([word for word in lemmatized_text if word not in stop_words])

    tokens = word_tokenize(stopworded_lemmatized_text)

    return tokens

In [293]:
df["Tokens"] = df["Text"].progress_apply(lambda x: preprocess(x))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [294]:
model = Word2Vec(df["Tokens"].values, min_count = 0)

In [295]:
df["MeanVector"] = df["Tokens"].progress_apply(lambda x: np.array([model.wv[word] for word in x]).mean(axis=0))

  0%|          | 0/10000 [00:00<?, ?it/s]

# Класификация

In [296]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

In [306]:
train = df[:9000].copy()
test = df[9000:].copy()

In [304]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(np.vstack(train["MeanVector"].values), train["Label"].values)

predictions = knn.predict(np.vstack(test["MeanVector"].values))

print(classification_report(test["Label"].values, predictions))

              precision    recall  f1-score   support

    business       0.55      0.37      0.44        46
     culture       0.78      0.90      0.83       126
   economics       0.75      0.90      0.82       148
      forces       0.72      0.81      0.76        75
        life       0.75      0.76      0.75       139
       media       0.82      0.65      0.73       156
     science       0.85      0.76      0.81       136
       sport       0.95      0.96      0.95       147
       style       0.71      0.77      0.74        13
      travel       0.23      0.21      0.22        14

    accuracy                           0.79      1000
   macro avg       0.71      0.71      0.71      1000
weighted avg       0.79      0.79      0.78      1000



# Второй подход

In [301]:
def few_popular(lst, count):
    values, counts = np.unique(lst, return_counts=True)
    return values[np.argsort(-counts)][:count]

In [344]:
# Средний вектор 10ти самых частых слов в документе
df["PopularVector"] = df["Tokens"].progress_apply(lambda x: np.array([model.wv[word] for word in few_popular(x, 10)]).sum(axis=0))

  0%|          | 0/10000 [00:00<?, ?it/s]

In [345]:
train = df[:9000].copy()
test = df[9000:].copy()

In [346]:
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(np.vstack(train["PopularVector"].values), train["Label"].values)

predictions = knn.predict(np.vstack(test["PopularVector"].values))

print(classification_report(test["Label"].values, predictions))

              precision    recall  f1-score   support

    business       0.47      0.15      0.23        46
     culture       0.68      0.81      0.74       126
   economics       0.64      0.82      0.72       148
      forces       0.56      0.55      0.55        75
        life       0.56      0.73      0.63       139
       media       0.76      0.58      0.65       156
     science       0.77      0.67      0.72       136
       sport       0.88      0.85      0.87       147
       style       0.56      0.38      0.45        13
      travel       0.33      0.07      0.12        14

    accuracy                           0.69      1000
   macro avg       0.62      0.56      0.57      1000
weighted avg       0.69      0.69      0.67      1000

